In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
from sklearn.model_selection import GroupKFold
from sklearn import metrics
import xgboost as xgb
from itertools import product


workbook = 'field_point.xlsx'
sheet = 'all'
df = pd.read_excel(workbook, sheet)

X = df.iloc[:, 5:-1].values  
Y = df.iloc[:, -1].values   


cell_size = 0.005  


geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")


gdf.to_file("SA.shp")
SA = gpd.read_file("SA.shp")


minx, miny, maxx, maxy = gdf.total_bounds
rows = int((maxy - miny) // cell_size + 3)
cols = int((maxx - minx) // cell_size + 3)

polygons = []
for i in range(rows):
    for j in range(cols):
        xmin = minx - 1.5 * cell_size + j * cell_size
        xmax = xmin + cell_size
        ymin = miny - 1.5 * cell_size + i * cell_size
        ymax = ymin + cell_size
        polygons.append(Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]))

grid_gdf = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")

grid_gdf['FID'] = np.arange(len(grid_gdf), dtype=int)

grid_shapefile = f"grid_{cell_size}.shp"
grid_gdf.to_file(grid_shapefile)


grid = gpd.read_file(grid_shapefile)


grid_df = pd.DataFrame(columns=[f'grid_{cell_size}'])
for idx, point in SA.iterrows():
    containing = grid[grid.contains(point.geometry)]
    if not containing.empty:
        field_value = int(containing.iloc[0]['FID'])
        grid_df.loc[idx] = [field_value]
    else:

        nearest = grid.loc[grid.distance(point.geometry).idxmin()]
        field_value = int(nearest['FID'])
        grid_df.loc[idx] = [field_value]


block = grid_df[f'grid_{cell_size}'].astype(int).values
n_groups = pd.Series(block).nunique()
if n_groups < 2:
    raise ValueError(f"Not enough unique grid blocks for CV (found {n_groups}).")
n_splits = 3 if n_groups >= 3 else 2
group_kfold = GroupKFold(n_splits=n_splits)
block_kfold = list(group_kfold.split(X, Y, block))


max_depth_grid      = list(range(1, 6))              
n_estimators_grid   = list(range(100, 1001, 100))    
learning_rate_grid  = [0.1, 0.2, 0.3]               
colsample_grid      = [i/10 for i in range(1, 10)] 


results = []
for md, ne, lr, cs in product(max_depth_grid, n_estimators_grid, learning_rate_grid, colsample_grid):
    r2s, maes, rmses = [], [], []
    for train_idx, test_idx in block_kfold:
        model = xgb.XGBRegressor(
            max_depth=md,
            n_estimators=ne,
            learning_rate=lr,
            colsample_bytree=cs,
            random_state=42,  
            n_jobs=1          
        )
        model.fit(X[train_idx], Y[train_idx])
        y_pred = model.predict(X[test_idx])
        y_true = Y[test_idx]

        r2s.append(metrics.r2_score(y_true, y_pred))
        maes.append(metrics.mean_absolute_error(y_true, y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(y_true, y_pred)))

    results.append({
        'max_depth': md,
        'n_estimators': ne,
        'learning_rate': lr,
        'colsample_bytree': cs,
        'R2_mean': float(np.mean(r2s)),
        'MAE_mean': float(np.mean(maes)),
        'RMSE_mean': float(np.mean(rmses))
    })


best = max(
    results,
    key=lambda d: (d['R2_mean'], -d['RMSE_mean'], -d['MAE_mean'])
)

print("Best hyperparameters (by mean R², with RMSE/MAE tie-breakers):")
print(f"  max_depth       = {best['max_depth']}")
print(f"  n_estimators    = {best['n_estimators']}")
print(f"  learning_rate   = {best['learning_rate']}")
print(f"  colsample_bytree= {best['colsample_bytree']}")
print("\nMean CV scores (3 folds):")
print(f"  R²   : {best['R2_mean']:.4f}")
print(f"  RMSE : {best['RMSE_mean']:.2f}")
print(f"  MAE  : {best['MAE_mean']:.2f}")
